In [4]:
from agential.cog.strategies.react.qa import ReActQAStrategy
from agential.cog.prompts.agents.react import (
    REACT_INSTRUCTION_HOTPOTQA,
)
from agential.cog.prompts.benchmarks.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT

import dotenv

dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


In [5]:
strat = ReActQAStrategy(llm=llm)

In [ ]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

out = strat.generate(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
)